# Svenskt Kvinnobiografiskt lexikon part 4
version part 3 - 1.10


* this [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%204.ipynb) 
  * [part 1](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) check Wikidata and SKBL
  * [part 2](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%202.ipynb) more queries etc.
  * [part 3](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb) statistics 


# Get Archives from SKBL metadata

source: type: Arkiv


# Wikidata

In [1]:
from datetime import datetime
now = datetime.now()
print("Last run: ", now)

Last run:  2020-10-02 11:44:08.339616


In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

querySKBL = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?wid)  ?skblid WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skblid
} order by ?skbl"""

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDSKBLtot = get_sparql_dataframe(endpoint_url, querySKBL)
WDSKBLtot["Source"] = "SKBL" 


In [3]:
WDSKBLtot.sort_index(inplace=True)  

In [4]:
WDSKBLtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1535 entries, 0 to 1534
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   wid     1535 non-null   object
 1   skblid  1535 non-null   object
 2   Source  1535 non-null   object
dtypes: object(3)
memory usage: 36.1+ KB


In [5]:
import csv  
import urllib3, json
http = urllib3.PoolManager() 

jsonURL = "https://skbl.se/sv/artikel/"
listNewItems =[]
for index,row in WDSKBLtot.iterrows():
    url = jsonURL + row["skblid"] + ".json"
    #print(url)
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))  
    new_item = dict()
    new_item['skblid'] = row["skblid"]
    new_item['wid'] = row["wid"]
    try:
        sources = data['source']  
        for s in sources:
            if s["type"] in "Arkiv":
                print(url)
                print (s["url"])
                print (s["description"])
    except:
        pass
    listNewItems.append(new_item)
print (len(listNewItems) ," antal poster")


https://skbl.se/sv/artikel/Sofiadrottning.json
http://www.kungahuset.se/monarkinhovstaterna/hovstaternasorganisation/kunglhusgeradskammaren/bernadotteskaarkivet.4.4ea495e313c19c119aa76c0.html
Bernadotteska arkivet (Hämtad 2017-11-23)
https://skbl.se/sv/artikel/LovisaUlrikadrottning.json
https://sok.riksarkivet.se/?Sokord=lovisa+ulrika&amp;EndastDigitaliserat=false&amp;AvanceradSok=False&amp;page=2&amp;postid=Arkis+A8FFB794-90D6-4E4F-8E25-A47DE6444603&amp;tab=post#tab
Drottning Lovisa Ulrika. Kungliga arkiv, Riksarkivet (Hämtad 2018-03-13)
https://skbl.se/sv/artikel/LovisaUlrikadrottning.json
https://sok.riksarkivet.se/?Sokord=Kammarkontorets+arkiv+r%C3%A4kningar&amp;EndastDigitaliserat=false&amp;AvanceradSok=False&amp;page=1&amp;postid=Arkis+B623BDC7-B88C-44AB-8D2E-47CC641D1900&amp;tab=post&amp;FacettState=undefined%3Ac%7C#tab
Räkningar. Kammarkontorets arkiv, Riksarkivet (Hämtad 2018-03-13)
https://skbl.se/sv/artikel/LovisaUlrikadrottning.json
https://sok.riksarkivet.se/?Sokord=lovisa

https://skbl.se/sv/artikel/NinniKronberg.json
https://www.naringslivshistoria.se/cfn-nyheter/engeltofta-lyfter-fram-historiska-agare/
Semper. *Centrum för Näringslivshistoria* (Hämtad 2017-07-04)
https://skbl.se/sv/artikel/SigridLeijonhufvud.json
https://sok.riksarkivet.se/?Sokord=sigrid+leijonhufvud&amp;EndastDigitaliserat=false&amp;AvanceradSok=False&amp;page=4&amp;postid=Arkis+3DD6E3AB-6637-45F6-B5B8-5C2F04F3825B&amp;s=Siv&amp;tab=post&amp;FacettState=undefined%3Ac%7C#tab
Sigrid Amalia Leijonhufvud. Kungliga biblioteket (Hämtad 2018-03-11)
https://skbl.se/sv/artikel/SigridLeijonhufvud.json
https://sok.riksarkivet.se/?Sokord=Carl+Gustaf+Tessin+&amp;EndastDigitaliserat=false&amp;AvanceradSok=False&amp;FacettLimits=qLer1g%3A0&amp;FacettState=undefined%3Ac%7C1UXH7A%3Ac%7C&amp;FacettFilter=arkis_ark_arkivinstitution%24Riksarkivet%3A&amp;f=True&amp;page=1#tab
Carl Gustaf Tessin. Riksarkivet (Hämtad 2018-03-11)
https://skbl.se/sv/artikel/GertrudLilja.json
https://sok.riksarkivet.se/?Sokord

https://skbl.se/sv/artikel/EmilieHogqvist.json
http://www.kb.se/samlingarna/Databaser/
Kungliga bibliotekets brevskrivarregister (Tillgängligt på KB)
https://skbl.se/sv/artikel/EmilieHogqvist.json
http://musikverket.se/musikochteaterbiblioteket/arkivregistret/
Arkivregistret, Musik- och Teaterbiblioteket (Hämtad 2017-12-19)
https://skbl.se/sv/artikel/EvaBonnier.json
https://skbl.se/sv/artikel/ElsaBrandstrom.json
https://arken.kb.se/SE-S-HS-L88
Elsa Brändströms samling. Kungliga biblioteket (Hämtad 2020-02-19)
https://skbl.se/sv/artikel/MargarethaKrook.json
http://www.dramaten.se/medverkande/rollboken/Person/3611
Margareta Krook. Arkivet Rollboken, Dramaten (Hämtad 2017-10-19)
https://skbl.se/sv/artikel/HarrietBosse.json
http://musikverket.se/musikochteaterbiblioteket/arkivregistret/#arkivreg-B
Bosse, Harriet. Musik- och teaterbiblioteket (Hämtad 2018-02-14)
https://skbl.se/sv/artikel/HarrietBosse.json
https://arken.kb.se/SE-S-HS-Acc2008-110
Papper efter Harriet Bosse. Kungl. Biblioteke

https://skbl.se/sv/artikel/HjordisHjalmarsdotterHjorth.json
https://sok.riksarkivet.se/nad?Sokord=hj%C3%B6rdis+hjalmarsdotter&EndastDigitaliserat=false&BegransaPaTitelEllerNamn=false&lk=Ladda+kategorier&AvanceradSok=False&typAvLista=Standard&page=1&postid=Arkis+4EFA1368-81B9-41D7-B54D-35FE05BBA57B&s=Siv&tab=post&FacettState=undefined%3Ac%7C#tab
Hjördis Hjalmarsdotters arkiv. Nordiska Museet (Hämtad 2018-03-01)
https://skbl.se/sv/artikel/NellWalden.json
http://www.landskrona.se/se-gora/kultur-noje/museerochkonsthall/museum/arkivochsamlingar/
Arkiv och samlingar, Landskrona museum (Hämtad 2017-05-29)
https://skbl.se/sv/artikel/BritaEgardt.json
http://www.folklivsarkivet.lu.se/om-folklivsarkivet/
Brita Egardts efterlämnade forskningsmaterial. Folklivsarkivet i Lund (Hämtad 2018-03-20)
https://skbl.se/sv/artikel/AmaliaLundeberg.json
https://sok.riksarkivet.se/?Sokord=Carl+August+Bagges+samling%2C&page=1&postid=Arkis+3EE28B99-9B89-11D5-A701-0002440207BB&tab=post&FacettState=undefined%3Ac%7C

https://skbl.se/sv/artikel/WendelaHebbe.json
https://www.ub.uu.se/hitta-i-vara-samlingar/arkiv/
Vendela Hebbe, Handskriftsenheten, Uppsala universitetsbibliotek (Hämtad 2020-02-23)
https://skbl.se/sv/artikel/EmilyNonnen.json
https://sok.riksarkivet.se/?Sokord=Nonnen&amp;EndastDigitaliserat=false&amp;AvanceradSok=False&amp;FacettFilter=arkis_ark_typ_facet%24Arkiv%3A&amp;FacettState=gG8Klw%3Ac%7Cundefined%3Ac%7C&amp;page=2&amp;postid=Arkis+873B2B1D-77A1-11D5-A6F2-0002440207BB&amp;tab=post#tab
Landeriet Liseberg och släkten Nonnen. Landsarkivet i Göteborg (Hämtad 2018-03-05)
https://skbl.se/sv/artikel/AlmaJohansson.json
https://sok.riksarkivet.se/?Sokord=Ragnvald&amp;FacettFilter=arkis_ark_typ_facet%24Alla%3A&amp;page=11&amp;postid=Arkis+281E1654-CD8A-11D5-841B-00D0B73E7A8B
Kvinnliga missionsarbetare, SE/RA/7840. Riksarkivet (Hämtad 2017-10-10)
https://skbl.se/sv/artikel/AlmaJohansson.json
http://www.armenocide.de/armenocide/ArmGenDE.nsf/0/DA7DFB02A0C594C5C1256AD700793C71?OpenDocument
The

...

https://skbl.se/sv/artikel/HarrietLowenhjelm.json
https://arken.kb.se/SE-S-HS-Acc2007-33
Handlingar rörande Harriet Löwenhjelm. Kungliga biblioteket (Hämtad 2020-02-25)
https://skbl.se/sv/artikel/HarrietLowenhjelm.json
http://www.foark.umu.se/samlingar/arkiv/148
Handskrift 148. Forskningsmaterial kring poeten och konstnären Harriet Löwenhjelm ... Forskningsarkivet, Umeå universitetsbibliotek (Hämtad 2017-08-16)
https://skbl.se/sv/artikel/HarrietLowenhjelm.json
https://www.alvin-portal.org/alvin/resultList.jsf?dswid=9961&p=1&searchType=STILL_IMAGE&sortString=relevance_sort_desc&noOfRows=10&af=%5B%5D&query=harriet+l%C3%B6wenhjelm&aq=%5B%5B%7B%22A_FQ%22%3A%22harriet+l%C3%B6wenhjelm%22%7D%5D%5D&aqe=%5B%5D
Harriet Löwenhjelm.  ALVIN - Plattform för digitala samlingar och digitaliserat kulturarv (Hämtad 2017-08-16)
https://skbl.se/sv/artikel/IdaNilsson.json
https://www.alvin-portal.org/alvin/view.jsf?dswid=-9099&searchType=EXTENDED&query=Ida+Nilssons+arkiv&aq=%5B%5B%7B%22A_FQ%22%3A%22Ida+Nil

https://skbl.se/sv/artikel/DeboravanderPlas0.json
https://archief.amsterdam/indexen
Gemeente Amsterdam Stadsarchief, Nederländerna Hervormd, Nieuwe kerk, Amsterdam: dop-, vigsel-och begravningsböcker
https://skbl.se/sv/artikel/DeboravanderPlas0.json
https://www.regionaalarchiefdordrecht.n
Regionalarchief Dordrecht, Nederländerna, Nederlands Hervormde Gemeente: dopböcker
https://skbl.se/sv/artikel/DeboravanderPlas0.json
http://digitalastadsarkivet.stockholm.se
Stockholms stadsarkiv Tyska församlingen, Nikolai församling: Register till kyrkböcker
https://skbl.se/sv/artikel/HarrietSundstrom0.json
https://sok.riksarkivet.se/?Sokord=harriet+sundstr%C3%B6m&FacettFilter=register_facet%24Brev%3A%7Cregister_facet%24Brev%2FRiksarkivets+enskilda+arkiv+och+samlingar%3A&FacettState=xXo49g%3Ao%7CvFUdZg%3Ac%7Cr_xM6A%3Ac%7C&page=1&f=True#tab
Brev från Harriet Sundström. Riksarkivets brevdatabas (Hämtad 2019-11-23)
https://skbl.se/sv/artikel/HarrietSundstrom0.json
https://sok.riksarkivet.se/?Sokord=har

https://skbl.se/sv/artikel/MathildaRanch.json
https://sok.riksarkivet.se/?Sokord=+FAMILJEN+RANCH%3B+VARBERG&EndastDigitaliserat=false&AvanceradSok=False&page=1&postid=Arkis+2FF8A65B-69F5-412C-8428-1B8C1A878954&s=Siv&tab=post&FacettState=undefined%3Ac%7C#tab
Familjen Ranch; Varberg, Länsmuseet Varberg  (hämtad 2019-10-28)
https://skbl.se/sv/artikel/ChristinaReginavonBirchenbaum.json
https://skbl.se/sv/artikel/HarrietClayhills0.json
https://skbl.se/sv/artikel/AnnaKlemetsdotterHogenskild0.json
https://skbl.se/sv/artikel/EvaHokerberg.json
https://sok.riksarkivet.se/?Sokord=eva+h%C3%B6kerberg&page=10&postid=Arkis+3EE299BD-9B89-11D5-A701-0002440207BB&tab=post&FacettState=undefined%3Ac%7C#tab
Eva Hökerbergs arkiv. Riksarkivet (Hämtad 2019-12-11)
https://skbl.se/sv/artikel/EvaHokerberg.json
https://sok.riksarkivet.se/?Sokord=Yrkeskvinnors++riksf%C3%B6rbund&EndastDigitaliserat=false&AvanceradSok=False&FacettFilter=arkis_ark_typ_facet%24Arkiv%3A&FacettState=gG8Klw%3Ac%7Cundefined%3Ac%7C&page=2&p

https://skbl.se/sv/artikel/KarnaAsker.json
https://skbl.se/sv/artikel/IngaBagge0.json
https://skbl.se/sv/artikel/CeciliaBaathHolmberg.json
https://sigtunastiftelsen.se/bibliotek-arkiv/
Bååth-Holmbergska samlingen. Sigtunastiftelsens arkiv (Hämtad 2020-04-17)
https://skbl.se/sv/artikel/ViraEklund.json
https://sok.riksarkivet.se/arkiv/T9FYFecRrH6d0G02H087k3
Folkpartiets kvinnoförbunds arkiv/Liberala kvinnors arkiv. Riksarkivet (Hämtad 2020-06-01)
https://skbl.se/sv/artikel/ViraEklund.json
https://sok.riksarkivet.se/arkiv/URUHvvlc9ac9hr6yAXT193
Vita bandet i Göteborg, föreningen. Region och Stadsarkivet Göteborg (Hämtad 2020-06-01)
https://skbl.se/sv/artikel/SaraHolmsten.json
https://sok.riksarkivet.se/amnesomrade?postid=Arkis+72e9081a-3dc5-4434-ac00-faa947da698c&infosida=amnesomrade-skatter&flik=1&vol=n&s=Balder
Evangeliska brödraförsamlingens arkiv. Riksarkivet (Hämtad 2020-03-24)
https://skbl.se/sv/artikel/GunhildAlmen.json
http://urn.kb.se/resolve?urn=urn:nbn:se:alvin:portal:record-70

https://skbl.se/sv/artikel/EliseSahlqvist.json
https://skbl.se/sv/artikel/GerdaHoglund0.json
https://soeg.kb.dk/permalink/45KBDK_KGL/1pioq0f/alma99122822723905763
Benna Moes efterladte papirer. Det Kongelige Bibliotek, Köpenhamn
https://skbl.se/sv/artikel/GerdaHoglund0.json
https://skbl.se/sv/artikel/BrittIngridMargaretaLundbohmReutersvard.json
http://urn.kb.se/resolve?urn=urn:nbn:se:alvin:portal:record-69269
Britt Lundbohm-Reutersvärds arkiv. Universitetsbiblioteket, Lunds universitet (Hämtad 2020-03-26)
https://skbl.se/sv/artikel/MariaZoega.json
https://skbl.se/sv/artikel/CorneliaHall0.json
https://skbl.se/sv/artikel/EbbaHultqvist.json
https://www.arkivdigital.se/
ArkivDigital (Abonnemang krävs)
https://skbl.se/sv/artikel/EbbaHultqvist.json
https://www.folklivsarkivet.lu.se/
Folklivsarkivet,Lunds universitet, Lund
https://skbl.se/sv/artikel/SigneGustafsson.json
https://skbl.se/sv/artikel/AurorePihl.json
https://sok.riksarkivet.se/?Sokord=Norrk%C3%B6pings+Norra+l%C3%A4roverk+f%C3%B6r+

In [6]:
if len(listNewItems) > 0:
    keys = listNewItems[0].keys()
    with open("SKBLAuthor_newfile.csv", "w", newline='') as SKBLfile:
        dict_writer = csv.DictWriter(SKBLfile, keys)
        dict_writer.writeheader()
        dict_writer.writerows(listNewItems)

    dfSKBLAuthorcsv = pd.read_csv("SKBLAuthor_newfile.csv", sep=",")   
    
else:
    print ("Ingen fil skapas inga nya poster") 
            


In [7]:
dfSKBLAuthorcsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1535 entries, 0 to 1534
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   skblid  1535 non-null   object
 1   wid     1535 non-null   object
dtypes: object(2)
memory usage: 24.1+ KB


## Store Authorname in WIkidata 

1. Step nr one is that we just add the author as a text string e.g.
  1. Example article UlrikaEleonoradydrottning has in metadata json see line 1 below  

    1. article_author_firstname: "Sari"
    1. article_author_lastname: "Nauman" 
    
I guess then we create a authorname string **Nauman, Sari**

Many of the authors has an Wikipedia article on Swedish Wikipedia --> then we could add the author as a Wikilink e.g.
Lisbeth Larsson [Q4961115](https://www.wikidata.org/wiki/Q4961115) is same as SKBL [articleauthor/Larsson, Lisbeth](https://skbl.se/en/articleauthor/Larsson%2C%20Lisbeth)  

See also [en:Wikipedia:Template:Talk:SKBL](https://en.wikipedia.org/wiki/Template_talk:SKBL#Store_author_in_Wikidata_and_use_in_the_SKBL_template)

In [8]:
dfSKBLAuthorcsv["publishedDate"] = dfSKBLAuthorcsv["publishedDate"].astype("datetime64")
dfSKBLAuthorcsv["Source"] = "SKBL"

KeyError: 'publishedDate'

In [ ]:
%matplotlib inline    
import matplotlib.pyplot as plt   


In [ ]:
plotPublishedAuthor =   dfSKBLAuthorcsv  
  
plotPublishedAuthor["Author"] = plotPublishedAuthor["article_author_firstname"] + " " + plotPublishedAuthor["article_author_lastname"]   
plotAuthor = plotPublishedAuthor["Author"].value_counts()[:20]
plotAuthor.plot( kind = 'pie') 
plt.title("SKBL authors -20")
plt.show()

In [ ]:
plotAuthor = plotPublishedAuthor["Author"].value_counts()[20:50]
plotAuthor.plot( kind = 'pie') 
plt.title("SKBL authors 20-50")
plt.show()

In [ ]:
#plotPublishedperMonth = dfSKBLAuthorcsv['publishedDate'].groupby(df['last_payout'].dt.to_period('M')).sum().plot(kind='bar')
#plotPublishedperMonth = dfSKBLAuthorcsv['publishedDate'].groupby([dfSKBLAuthorcsv.publishedDate.year,dfSKBLAuthorcsv.publishedDate.month].agg('count'))
plotPublishedperMonth = dfSKBLAuthorcsv['publishedDate'].groupby(dfSKBLAuthorcsv.publishedDate.dt.to_period("M")).agg('count')
plotPublishedperMonth.plot( kind = 'bar') 
plt.title("SKBL articles published per month")
plt.show()

In [ ]:
plotPublishedperMonth

In [ ]:
plotPublishedperMonth.plot( kind = 'pie') 
plt.title("SKBL articles published per month")
plt.show()

In [ ]:
pd.set_option('display.max_rows', None) 
# Sort latest first 
#dfSKBLAuthorcsv.sort_values(by=['publishedDate'], ascending=[False])

In [ ]:
from IPython.display import display, HTML  
plotPublishedAuthor['Link'] = plotPublishedAuthor['skblid'].apply(lambda x: "https://skbl.se/sv/artikel/{0}".format(x))  
plotPublishedAuthor.style.set_properties(subset=['publishedDate'], **{'width': '200px'}) 
plotPublishedAuthor = plotPublishedAuthor.sort_values(by=['publishedDate'], ascending=[False])
plotPublishedAuthor = plotPublishedAuthor.reset_index(drop=True)  

# de sista 10 publiserade biografier 
plotPublishedAuthor[:10]

## Compare SBL
SBL started in 1918 and is published by the [National Archives](https://sok.riksarkivet.se/sbl/Start.aspx?lang=en) in Sweden. They have
* No API
* focus on publish on paper
* dont have structured data
* no understanding of [Linked data](https://vimeo.com/36752317)
* we have before manually and doing webscrabing populated [Wikidata P3217](https://www.wikidata.org/wiki/Property_talk:P3217) now I guess we wait on SBL for delivering data as data see question asked [236883](https://phabricator.wikimedia.org/T236883) Oct 30 2019, 11:03 AM - no official answer so far and The Swedish National Archives has no helpdesk system were you get an unique helpdesk number ([some action was promised aug 2018 by Rolf Källman](https://twitter.com/salgo60/status/1030269242296414208?s=20) nothing heard)


In [ ]:
# SPARQL 
querySBL = """SELECT distinct (REPLACE(STR(?item), ".*Q", "Q") AS ?wid)  ?sblid  ?genderLabel  WHERE {
?item wdt:P31 wd:Q5.
OPTIONAL {?item wdt:P21 ?gender}
?item wdt:P3217 ?sblid
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} order by ?sblid """


WDSBLtot = get_sparql_dataframe(endpoint_url, querySBL)
WDSBLtot["Source"] = "SBL" 
WDSBLtot.info()

In [ ]:
plotSBLgender = WDSBLtot["genderLabel"].value_counts()
plotSBLgender.plot( kind = 'pie') 
plt.title("Riksarkivet SBL male/female")
plot = plotSBLgender.plot.pie( \
                   autopct='%1.0f%%', \
                   shadow=True, startangle=0)
plt.title("SBL male vs. female articles 1918 - " + now.strftime('%Y %m %d'))

plt.show()

In [ ]:
WDSBLtot.info()

In [ ]:
WDSBLtot.duplicated(subset=['wid']).sum()

In [ ]:
plotSBLgender

In [ ]:
plotSBLfemale = WDSBLtot[WDSBLtot["genderLabel"] == "female"] 
#plotSBLfemale["birth"] = plotSBLfemale["birth"].astype("datetime64")
plotSBLfemale.info() 

In [ ]:
#Merge dfSKBLAuthorcsv and plotSBLfemale 


In [ ]:
end = datetime.now()
print("Ended: ", end)

In [ ]:
dfmerge = pd.merge(plotSBLfemale, plotPublishedAuthor,how='outer', on='wid',indicator=True)

In [ ]:
dfmerge.info()

In [ ]:
dfmerge["_merge"].value_counts()

In [ ]:
# both is number in bioth SBL and SKBL   
# right_only is SBL
# left_only is SKBL

In [ ]:
dfmerge['_merge'] = dfmerge['_merge'].str.replace('left_only','SBL_only').str.replace('right_only','SKBL_only')
dfmerge[0:10]

In [ ]:
mergeCheck = dfmerge[['wid', 'sblid','skblid','publishedDate','_merge']] 
mergeCheck.info() 

In [ ]:
mergeCheckNumber = mergeCheck["_merge"].value_counts()
mergeCheckNumber.plot( kind = 'pie')  
plot = mergeCheckNumber.plot.pie( \
                   autopct='%1.0f%%', \
                   shadow=True, startangle=0)
plt.title("SBL vs SKBL female articles " + now.strftime('%Y %m %d'))
plt.show()


In [ ]:
mergeCheckNumber

In [ ]:
#Profiles missing in SKBL that SBL has written = left_only 
WDSBLonly = mergeCheck[mergeCheck["_merge"] == "SBL_only"].copy() 

In [ ]:
WDSBLonly.info()

In [ ]:
WDSBLonly["sbl"] = "https://sok.riksarkivet.se/Sbl/Presentation.aspx?id=" + WDSBLonly["sblid"] 
pd.set_option('column_space', 20)
pd.set_option('max_colwidth', 300)

WDSBLonly[["wid","sblid","sbl"]][:10]

### Same query in Wikidata 
* Find all women [in SBL but not in SKBL](https://w.wiki/bFw) = 196 women
* Find all women [in SBL but not in SKBL and have an en:Wikipedia article](https://w.wiki/bFy) = 128 
### Wikipedia articles linking SKBL 
* sv:Wikipedia [usage template Authority](https://sv.wikipedia.org/wiki/Kategori:Wikipediaartiklar_med_identifierare_fr%C3%A5n_SKBL) > 1410
* en:Wikipedia [usage template SKBL](https://en.wikipedia.org/wiki/Category:SKBL_template_using_Wikidata_property_P4963) > 686 articles

* SPARQL sv
** Swedish
** English

In [ ]:
WDSBLonly.info()

In [ ]:
WDSBLonly.drop_duplicates(subset ="wid", keep = False, inplace = True)
WDSBLonly.info() 

In [ ]:
end = datetime.now()
print("Ended: ", end)